In [0]:
%run "/Workspace/exec-ccmd-ussocom-personnel-r/Data Engineering/DMDC/J1 SEP Data Pipelines/pipelines/pipeline_builder_notebooks/pipeline_builder_v2.8.3"

In [0]:
# rank_pipeline.py
# PySpark 3.2 • Python 3.8 • Delta Lake

from pprint import pprint
from pyspark.sql import functions as F, Window, SparkSession, DataFrame
# from medallion_pipeline import build_pipeline

In [0]:
# ───────────────────────── Bronze rules ─────────────────────────
bronze_rules = {
    # Keep columns by listing them as keys; add checks as list items
    # ppln_cd: length 2
    "ppln_cd": [
        F.col("ppln_cd").isNotNull(),
        F.length("ppln_cd").between(2, 2),
    ],
    
    "pn_ssn_id_hash": [
        F.col("pn_ssn_id_hash").isNotNull()
    ],
    
    # pri_svc_occ_cd: 1..10 chars
    "pri_svc_occ_cd": [
        F.col("pri_svc_occ_cd").isNotNull(),
        F.length("pri_svc_occ_cd").between(1, 10),
    ],
    # otpt_file_dt: YYYYMMDD and a valid date
    "otpt_file_dt": [
        F.col("otpt_file_dt").isNotNull(),
    ],
    # pgrd_id: 1..2 digits
    "pgrd_id": [
        F.col("pgrd_id").isNotNull(),
        F.length("pgrd_id").between(1, 2),
        F.col("pgrd_id").rlike(r"^\d{1,2}$"),
    ],
    # Keep svc_cd but do NOT require 'N' in Bronze
    "svc_cd": [
        F.col("svc_cd").isNotNull(),
    ],
}

In [0]:
# ─────────────────────── Silver transform & rules ───────────────────────
def silver_transform(
    spark: SparkSession,
    bronze_df: DataFrame,
    prior_silvers: dict[str: DataFrame]
) -> DataFrame:
    """
    Transform Bronze → Silver.
    Filters to Navy here, derives rank/personnel,
    and adds analysis columns.
    """
    return (
        bronze_df
        .filter(F.col("svc_cd") == "N")  # Navy filter lives in the transform
        .withColumn("source_date", F.col("otpt_file_dt"))
        .withColumn("pay_grade", F.col("pgrd_id").cast("int"))
        .withColumn(
            "personnel_type",
            F.when(F.substring("ppln_cd", -1, 1) == "E", "Enlisted")
             .when(F.substring("ppln_cd", -1, 1) == "W", "Warrant Officer")
             .when(F.substring("ppln_cd", -1, 1) == "O", "Officer")
             .otherwise("Unknown"),
        )
        .withColumn("rank_code", F.concat(F.substring("ppln_cd", -1, 1), F.col("pgrd_id")))
        .withColumn(
            "specialty",
            F.when(F.col("pri_svc_occ_cd").like("113%"), "SEAL Officer")
             .when(F.col("pri_svc_occ_cd").like("715%"), "SEAL WO")
             .when(F.col("pri_svc_occ_cd").like("717%"), "SWCC WO")
             .when(F.col("pri_svc_occ_cd").like("SO%"), "SEAL Enlisted")
             .when(F.col("pri_svc_occ_cd").like("SB%"), "SWCC Enlisted")
             .otherwise("No Group"),
        )
        .withColumn(
            "rank_order",
            F.when(F.col("rank_code").like("E%"), F.col("pgrd_id").cast("int"))
             .when(F.col("rank_code").like("W%"), 9 + F.col("pgrd_id").cast("int"))
             .when(F.col("rank_code").like("O%"), 14 + F.col("pgrd_id").cast("int"))
             .otherwise(25),
        )
        .withColumn(
            "years_in_service",
            F.round(F.datediff(F.current_date(), F.col("source_date")) / F.lit(365.25), 2),
        )
        .withColumn("service_branch", F.lit("Navy"))
        .withColumn("is_sof", F.lit(True))
        .withColumn("processed_timestamp", F.current_timestamp())
    )


SPECIALTIES = ["SEAL Officer", "SEAL WO", "SWCC WO", "SEAL Enlisted", "SWCC Enlisted", "No Group"]
PERSONNEL = ["Enlisted", "Warrant Officer", "Officer", "Unknown"]

silver_rules = {
    "pn_ssn_id_hash": [F.col("pn_ssn_id_hash").isNotNull()],
    "specialty": [F.col("specialty").isin(SPECIALTIES)],
    "personnel_type": [F.col("personnel_type").isin(PERSONNEL)],
    "rank_code": [F.col("rank_code").isNotNull()],
    "pay_grade": [F.col("pay_grade").between(1, 10)],
    "service_branch": [F.col("service_branch") == F.lit("Navy")],
    "is_sof": [F.col("is_sof") == F.lit(True)],
    "source_date": [F.col("source_date").isNotNull()],
    "rank_order": [F.col("rank_order").between(1, 25)],
    "years_in_service": [
        (F.col("years_in_service").isNull())
        | F.col("years_in_service").between(0.0, 50.0)
    ],
    "processed_timestamp": [F.col("processed_timestamp").isNotNull()],
}

In [0]:
# ─────────────────────── Gold transform & rules ───────────────────────
def gold_transform(
    spark: SparkSession,
    silver_dfs: dict[str, DataFrame]
) -> DataFrame:
    """
    Build a current-record view per person and derive readiness fields.
    """
    w = Window.partitionBy("pn_ssn_id_hash").orderBy(
        F.desc("source_date"),
        F.desc("processed_timestamp")
    )
    return (
        silver_dfs['silver_main']
        .withColumn("rn", F.row_number().over(w))
        .filter(F.col("rn") == 1)
        .drop("rn")
        .withColumn(
            "sof_readiness_score",
            F.when(F.col("specialty") == "No Group", F.lit(0.0))
             .otherwise(F.col("pay_grade").cast("double") * F.lit(10.0)),
        )
        .withColumn(
            "deployment_status",
            F.when(F.col("years_in_service") < 1.0, "Training")
             .when(F.col("sof_readiness_score") >= 100.0, "Ready")
             .otherwise("Ready"),
        )
        .withColumn(
            "rank_progression_potential",
            F.when(F.col("years_in_service") < 10, "High")
             .when(F.col("years_in_service") < 20, "Medium")
             .otherwise("Low"),
        )
        .withColumn("is_current_record", F.lit(True))
        .withColumn("last_updated", F.current_date())
        .withColumnRenamed("specialty", "current_specialty")
        .withColumnRenamed("rank_code", "current_rank_code")
        .withColumnRenamed("pay_grade", "current_pay_grade")
    )


gold_rules = {
    "pn_ssn_id_hash": [F.col("pn_ssn_id_hash").isNotNull()],
    "current_specialty": [F.col("current_specialty").isin(SPECIALTIES)],
    "current_rank_code": [F.col("current_rank_code").isNotNull()],
    "current_pay_grade": [F.col("current_pay_grade").between(1, 10)],
    "rank_order": [F.col("rank_order").between(1, 25)],
    "personnel_type": [F.col("personnel_type").isin(PERSONNEL)],
    "years_in_service": [
        (F.col("years_in_service").isNull())
        | F.col("years_in_service").between(0.0, 50.0)
    ],
    "sof_readiness_score": [F.col("sof_readiness_score").between(0.0, 150.0)],
    "deployment_status": [
        F.col("deployment_status").isin("Ready", "Training", "Medical", "Administrative")
    ],
    "rank_progression_potential": [
        F.col("rank_progression_potential").isin("High", "Medium", "Low")
    ],
    "is_current_record": [F.col("is_current_record") == F.lit(True)],
    "last_updated": [F.col("last_updated").isNotNull()],
}

In [0]:
# ─────────────────────────── Build pipeline ───────────────────────────
write_schema = 'exec_ccmd_ussocom_personnel_restricted_analytics'

builder: PipelineBuilder = (
    PipelineBuilder(
        spark=spark,
        schema=write_schema,
        min_bronze_rate=95.0,
        min_silver_rate=98.0,
        min_gold_rate=99.0,
        verbose=True,
    )
    .with_bronze_rules(
        name="bronze_main",
        rules=bronze_rules,
        incremental_col="otpt_file_dt",  # must be DateType in your bronze input
    )
    .add_silver_transform(
        name="silver_main",
        source_bronze="bronze_main",
        transform=silver_transform,
        rules=silver_rules,
        table_name="silver_navy_sof_personnel_3",
        watermark_col="source_date",  # must be DateType in persisted Silver table
    )
    .with_silver_rules(
        name="silver_old",
        rules=silver_rules,
        table_name="silver_navy_sof_personnel_3",
        schema='exec_ccmd_ussocom_personnel_restricted_analytics'
    )
    .with_gold_rules(
        name='gold_old',
        rules=gold_rules,
        table_name="gold_navy_sof_ranked_personnel_3",
        schema='exec_ccmd_ussocom_personnel_restricted_analytics',
        optional=True  # table may not exist yet (e.g. first run); use empty DataFrame
    )
    .add_gold_transform(
        name="gold_main",
        transform=gold_transform,
        rules=gold_rules,
        table_name="gold_navy_sof_ranked_personnel_4",
    )
)

pipe: PipelineRunner = builder.to_pipeline()

15:49:25 - PipelineRunner - INFO - 🔧 PipelineBuilder initialized (schema: exec_ccmd_ussocom_personnel_restricted_analytics)
15:49:25 - PipelineRunner - INFO - ✅ Added Bronze step: bronze_main
15:49:25 - PipelineRunner - INFO - ✅ Added Silver step: silver_main (source: bronze_main)
15:49:52 - PipelineRunner - INFO - ✅ Added Silver step (validation-only): silver_old
15:49:53 - PipelineRunner - INFO - ✅ Added Gold step (validation-only): gold_old
15:49:53 - PipelineRunner - INFO - 🔍 Auto-inferred source_silvers: ['silver_main', 'silver_old']
15:49:53 - PipelineRunner - INFO - ✅ Added Gold step: gold_main (sources: ['silver_main', 'silver_old'])
15:49:53 - PipelineRunner - INFO - ✅ Pipeline validation passed
15:49:53 - PipelineRunner - INFO - Starting dependency analysis with strategy: hybrid
15:49:53 - PipelineRunner - INFO - Dependency analysis completed in 0.00s
15:49:53 - PipelineRunner - INFO - 📋 Execution order (5 steps): bronze_main → silver_main → silver_old → gold_old → gold_main
15:49:53 - PipelineRunner - INFO - 🚀 Pipeline built successfully with 1 bronze, 2 silver, 2 gold steps